In [14]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Reusable classes
import sys
sys.path.append("/home/jovyan/work/upload/ml-experiments/sc_wrangling")

# Data path
import pathlib
FOLDER = pathlib.Path("/home/jovyan/work/upload/")
TABLE_PATH = FOLDER / 'data/pickle/df_columns_labels.pkl'
LAW2VEC_PATH = FOLDER / 'data/law2vec/Law2Vec.200d.txt'

# compress the trained model
import pickle
SAVED_MODEL_PATH = pathlib.Path('/home/jovyan/work/upload/data/trained_models')

# Math stuff
import numpy as np
from numpy.core.records import ndarray

# Manage ML lifecycle
import mlflow
from mlflow import log_params, set_tags, log_metrics, log_artifact
MLFLOW_TRACKING_URI = 'http://srv.meaningfy.ws:8989'

# Data visualisation
import pandas as pd
from pandas import Series

# Vectorize algotrithm
from gensim.models import Word2Vec, KeyedVectors

# Support Vector Classifier Algorithm
from sklearn.svm import SVC
# :gamma: is a parameter for non linear hyperplanes. The higher the gamma
#         value it tries to exactly fit the training data set.
GAMMA = 1
# :C: is the penalty parameter of the error term. It controls the trade
#     off between smooth decision boundary and classifying the training points correctly.
#     !!! Increasing C values may lead to overfitting the training data. !!!
C = 5

# Intermediate steps of the pipeline must be ‘transforms’,
# that is, they must implement fit and transform methods.
from sklearn.pipeline import Pipeline

# Splitting data into train and test
from sklearn.model_selection import train_test_split
RANDOM_STATE = 42
# Where "train" contains 70% of data and "test" - 30%
TEST_SIZE = 0.3
SHUFFLE = True

# Mean embedding function
from mean_vectorizer import MeanEmbeddingVectorizer

# Metrics Evaluation Methods
from evaluation_metrics import model_evaluation_metrics

# Transform DataFrame to dictionary
from dictionary_transformation import series_pair_to_dict

In [2]:
# Read data
df = pd.read_pickle(TABLE_PATH)
# Independent data
columns = df['Concatenated Data (clean)']
# Label data
category = df['Type of measure']

## law2vec model

In [3]:
# Load a word2vec model stored in the C *text* format.
model = KeyedVectors.load_word2vec_format(LAW2VEC_PATH, binary=False)
l2v_dict = {w: vec for w, vec in zip(model.wv.index2word, model.wv.syn0)}

In [4]:
# Use pipes to implement steps of fit and transform method
svm_l2v = Pipeline([
    # Add the words we want to mean
    ("law2vec vectorizer", MeanEmbeddingVectorizer(l2v_dict)),
    # Use SVC algorithm
    ("SVM", SVC(gamma=1, C=5))])

## Train SVM Model

In [5]:
# split data into test and train sets
X_train, X_test, y_train, y_test = train_test_split(
    columns, category, random_state=RANDOM_STATE, test_size=TEST_SIZE, shuffle=SHUFFLE)

In [6]:
# Fit SVM model with out train data
svm_l2v.fit(X_train, y_train)

Pipeline(steps=[('law2vec vectorizer',
                 <mean_vectorizer.MeanEmbeddingVectorizer object at 0x7f9209bd12e0>),
                ('SVM', SVC(C=5, gamma=1))])

### Save the model into pickle file

In [7]:
with open(SAVED_MODEL_PATH / 'type_of_measure_svm_law2vec.pkl', 'wb') as pickle_model:
    pickle.dump(svm_l2v, pickle_model)

In [9]:
# Model Evaluation
svm_l2v.score(X_train, y_train)

0.963302752293578

In [10]:
# SVM prediction based on test data
prediction = svm_l2v.predict(X_test)
prediction

array([0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 3, 0, 1, 1, 0, 2, 0,
       0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 2, 0, 0, 2, 0, 2, 2, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0,
       0, 2, 0, 0, 5, 0, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       2, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 3, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 2, 0, 0, 1, 0, 0, 0, 2, 0, 2, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 2, 2, 0, 3, 0, 0, 2, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0,
       1, 3, 0, 0, 1, 0, 4, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

## Evaluation

In [11]:
evaluation = model_evaluation_metrics(y_test, prediction)
evaluation

,Metric Labels,Metric values
0,Accuracy,0.818505
1,Precision,0.682853
2,Recall,0.456966
3,F1-Score,0.500542
4,Mean Absolute Error,0.373665
5,Mean Squared Error,1.021352


In [12]:
transformation = series_pair_to_dict(evaluation, 'Metric Labels', 'Metric values')
transformation


{'Accuracy': 0.8185053380782918,
 'Precision': 0.6828529818954893,
 'Recall': 0.4569662509961017,
 'F1-Score': 0.5005423698264986,
 'Mean Absolute Error': 0.3736654804270463,
 'Mean Squared Error': 1.0213523131672597}

In [15]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(experiment_name="Type of measure (SVM, Law2Vec)")

with mlflow.start_run():

    parameters = {"Language model": 'law2vec',
                  "Random state": RANDOM_STATE,
                  "Test size": TEST_SIZE,
                  "Shuffle": SHUFFLE,
                  "Gamma": GAMMA,
                  "C": C}
    log_params(parameters)

    log_metrics(transformation)

    log_artifact('/home/jovyan/work/upload/data/trained_models/type_of_measure_svm_law2vec.pkl')